In [1]:
import pandas as pd
import logging
from typing import Dict, Any, List, Union, Optional

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [37]:
#pd.set_option('display.max_columns', 65)

In [3]:
file_path = '/home/stanoo/dcrt/data'
df = pd.read_csv(f'{file_path}/processed_23-24_df.csv')

/tmp/ipykernel_4463/2985956275.py:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{file_path}/processed_23-24_df.csv')


In [58]:
problematic_columns = df.columns[df.apply(lambda col: col.astype(str).str.contains(r"[\u2019]").any())]

In [5]:
#stations
stations = df.court.unique()

In [7]:
hc_stations = ['Marsabit', 'Embu', 'Chuka', 'Muranga']

In [8]:
sub_df = df[df['court'].isin(hc_stations)]

In [9]:
sub_df.head()

,court,date_dd,date_mon,date_yyyy,caseid_type,caseid_no,filed_dd,filed_mon,filed_yyyy,original_court,...,broad_case_type,case_number,concluded,registered,merit_category,age,timely,adjourned,adjournable,transfer
170573,Embu,19,Jul,2023,HCCC,6,4,Nov,2018,NaN,...,Civil Matter,Embu/HCCC/6/2018,0,False,NaN,1718,False,0,1,False
170574,Embu,25,Sep,2023,HCFP&A,640,11,Dec,2014,NaN,...,Probate Administration,Embu/HCFP&A/640/2014,0,False,NaN,3210,False,0,1,False
170575,Embu,6,Jul,2023,HCFP&A,329,8,Aug,2011,NaN,...,Probate Administration,Embu/HCFP&A/329/2011,0,False,NaN,4350,False,0,1,False
170576,Embu,5,Jul,2023,HCCRC,3,24,Feb,2020,NaN,...,Murder,Embu/HCCRC/3/2020,1,False,Merit Resolution,1227,False,0,1,False
170577,Embu,6,Jul,2023,HCFP&A,E016,12,Jul,2022,NaN,...,Probate Administration,Embu/HCFP&A/E016/2022,0,False,NaN,359,False,0,1,False


In [56]:
df['judge_1'] = df['judge_1'].astype(str).str.replace(r"[\u2019]", "'")

In [4]:
df[(df['court'] == 'Iten') & (df['concluded'] == 1) & (df['broad_case_type']=='Murder')].groupby('outcome')['concluded'].count().sort_values(ascending=False).reset_index(name='count')

,outcome,count
0,Order Issued - Case Closed,3
1,Consolidated- Case Closed,1
2,Judgment Delivered- Case Closed,1
3,Terminated,1


In [44]:
# # drop the columns  'set_date', 'delivery_date', 'time_taken_days', 'judgment_status', 'delivery_category' from milimani_df
# df = df.drop(['set_date', 'delivery_date', 'time_taken_days', 'judgment_status', 'delivery_category'], axis=1)

In [45]:
def create_date_column(df: pd.DataFrame, column_names: List[str], new_col: str) -> pd.DataFrame:
    """
    Creates a new date column in the DataFrame by concatenating the values of three specified columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column_names (List[str]): A list of three column names to be concatenated [year, month, day].
        new_col (str): The name of the new date column to be created.

    Returns:
        pd.DataFrame: The DataFrame with the new date column added.

    Raises:
        ValueError: If the input list doesn't contain exactly three column names or if columns are missing.
    """
    if len(column_names) != 3:
        raise ValueError("column_names must contain exactly three elements: [year, month, day]")

    year_col, month_col, day_col = column_names

    # Check if all required columns exist in the DataFrame
    missing_columns = set(column_names) - set(df.columns)
    if missing_columns:
        raise ValueError(f"Missing columns in DataFrame: {', '.join(missing_columns)}")

    # Create copies to avoid SettingWithCopyWarning
    df = df.copy()

    try:
        # Convert year and day columns to integers
        df[year_col] = df[year_col].astype(float).astype(int)
        df[day_col] = df[day_col].astype(float).astype(int)

        # Concatenate the columns to create a date string
        df[new_col] = (df[year_col].astype(str) + '-' + 
                       df[month_col].astype(str) + '-' + 
                       df[day_col].astype(str))

        # Convert to datetime
        df[new_col] = pd.to_datetime(df[new_col], errors='coerce')

        # Log information about the conversion
        valid_dates = df[new_col].notna().sum()
        logger.info(f"Created new date column '{new_col}'. Valid dates: {valid_dates}/{len(df)}")

    except Exception as e:
        logger.error(f"Error creating date column: {str(e)}")
        raise

    return df

In [48]:
df = create_date_column(df.copy(), ['next_dd', 'next_mon', 'next_yyyy'], 'next_date')

/tmp/ipykernel_121153/4091687294.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[new_col] = pd.to_datetime(df[new_col], errors='coerce')
2024-11-04 09:16:16,989 - INFO - Created new date column 'next_date'. Valid dates: 427065/508802


In [49]:
# where the sheduled/next date is outside the evaluation period
cutoff_date = pd.Timestamp('2024-06-30') 

In [15]:
def determine_judgment_scheduling(df, cutoff_date):
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]
    
    # Preprocessing: Filter and sort the DataFrame upfront
    df_filtered = df[df['outcome'].isin(judgment_date_set_outcomes + judgment_delivered_outcomes)]
    df_filtered = df_filtered.sort_values(by=['case_number', 'activity_date'])
    
    # Initialize columns
    df['judgment_status'] = 'Not Scheduled'
    df['set_date'] = pd.NaT
    df['delivery_date'] = pd.NaT
    df['delivery_category'] = ''
    
    # Filter rows with judgment set outcomes and valid schedule dates
    judgment_set_rows = df_filtered[df_filtered['outcome'].isin(judgment_date_set_outcomes) & 
                                    (df_filtered['next_date'] <= cutoff_date)]
    
    # For each case, find the earliest set date
    earliest_schedule = judgment_set_rows.groupby('case_number').first().reset_index()
    
    # Create dictionaries to map case numbers to their schedule dates and statuses
    case_to_set_date = dict(zip(earliest_schedule['case_number'], earliest_schedule['next_date']))
    case_to_status = {case: 'Scheduled' for case in earliest_schedule['case_number']}
    
    # Update the result dataframe with schedule information
    df['set_date'] = df['case_number'].map(case_to_set_date)
    df['judgment_status'] = df['case_number'].map(case_to_status).fillna('Not Scheduled')
    df['delivery_category'] = df['case_number'].map(case_to_status).fillna('')
    
    # Filter rows with judgment delivered outcomes
    judgment_delivered_rows = df_filtered[df_filtered['outcome'].isin(judgment_delivered_outcomes)]
    
    # Find the first delivery date after set date
    for case_number, group in earliest_schedule.groupby('case_number'):
        set_date = group['next_date'].values[0]
        delivery = judgment_delivered_rows[(judgment_delivered_rows['case_number'] == case_number) & 
                                           (judgment_delivered_rows['activity_date'] >= set_date)]
        
        if not delivery.empty:
            delivery_date = delivery.iloc[0]['activity_date']
            df.loc[df['case_number'] == case_number, 'delivery_date'] = delivery_date
            df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delivered'
            
            if delivery_date <= set_date:
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'On Time'
            else:
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'Delayed'
        else:
            earlier_delivery = judgment_delivered_rows[(judgment_delivered_rows['case_number'] == case_number) & 
                                                       (judgment_delivered_rows['activity_date'] < set_date)]
            if earlier_delivery.empty:
                if cutoff_date >= set_date:
                    df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delayed'
                    df.loc[df['case_number'] == case_number, 'delivery_category'] = 'Delayed'
            else:
                df.loc[df['case_number'] == case_number, 'delivery_date'] = earlier_delivery.iloc[0]['activity_date']
                df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delivered'
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'On Time'
    
    return df[df['set_date'].notna()]


In [50]:
judgement_df = determine_judgment_scheduling(df, cutoff_date)

UFuncTypeError: ufunc 'less_equal' did not contain a loop with signature matching types (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.StrDType'>) -> None

In [54]:
def get_on_time_delivery_proportions(scheduled_cases):
    # Get the final status for each case
    final_status = scheduled_cases.groupby(['court', 'case_number']).last().reset_index()
    
    # Group by court and calculate statistics
    court_stats = final_status.groupby('court').agg({
        'case_number': 'count',
        'delivery_category': lambda x: (x == 'On Time').sum()
    }).rename(columns={
        'case_number': 'total_scheduled',
        'delivery_category': 'delivered_on_time'
    })
    
    # Calculate the proportion
    court_stats['proportion_on_time'] = court_stats['delivered_on_time'] / court_stats['total_scheduled']
    
    return court_stats

In [55]:
proportions2 = get_on_time_delivery_proportions(judgement_df)

In [56]:
proportions2.to_csv(f'{file_path}/reports/scheduled_judgement.csv', index=False)

In [58]:
def calculate_judgment_time(df):
    # Define the outcomes representing judgment date set and judgment delivered
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]

    # Create a copy of the original DataFrame to avoid modifying it
    result = df.copy()

    result['time_taken_days'] = pd.NaT

    # Group by case_number
    grouped = df.groupby('case_number')

    for case_number, group in grouped:
        # Find the minimum judgment set date
        set_date = group[group['outcome'].isin(judgment_date_set_outcomes)]['activity_date'].min()
        
        # Find the maximum judgment delivered date
        delivery_date = group[group['outcome'].isin(judgment_delivered_outcomes)]['activity_date'].max()

        # If there is no set date but there is a delivery date and an earliest "Judgement" entry
        earliest_judgement_date = group[group['comingfor'] == "Judgement"]['activity_date'].min()
        if pd.isna(set_date) and pd.notna(delivery_date) and pd.notna(earliest_judgement_date):
            # Get dates for the same case before the earliest "Judgement"
            previous_dates = group[(group['activity_date'] < earliest_judgement_date) & (group['activity_date'] >= group['activity_date'].min())]['activity_date']

            if not previous_dates.empty:
                set_date = previous_dates.max()

        # If both dates are available, calculate the time taken
        if pd.notna(set_date) and pd.notna(delivery_date):
            time_taken_days = (delivery_date - set_date).days
            result.loc[result['case_number'] == case_number, 'set_date'] = set_date
            result.loc[result['case_number'] == case_number, 'delivery_date'] = delivery_date
            result.loc[result['case_number'] == case_number, 'time_taken_days'] = time_taken_days

    return result


In [ ]:
time_taken = calculate_judgment_time(judgement_df)

In [64]:
delivered_df = time_taken[time_taken['judgment_status']=='Delivered']

In [65]:
delivered_status = delivered_df.groupby(['court', 'case_number']).last().reset_index()

In [67]:
delivered_status['within_60_days'] = delivered_status['time_taken_days'] <= 60
# group the data by court and calculate the proportion of cases within 60 days
within_60_days = delivered_status.groupby('court').agg({
    'case_number': 'count',
    'within_60_days': lambda x: (x == True).sum()
}).rename(columns={
    'case_number': 'total_scheduled',
    'within_60_days': 'delivered_within_60_days'
})

within_60_days['proportion_within_60_days'] = within_60_days['delivered_within_60_days'] / within_60_days['total_scheduled']

In [68]:
within_60_days.to_csv(f'{file_path}/reports/delivered_within_60_days.csv')

In [69]:
def get_case_resolution_proportions(df, case_type_timelines):
    # Convert date columns to datetime
    date_columns = ['filed_date', 'activity_date']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
    
    # Function to check if a case is resolved within timeline
    def is_resolved_within_timeline(row):
        timeline = case_type_timelines.get(row['broad_case_type'], float('inf'))
        return row['concluded'] == 1 and row['age'] <= timeline

    # Group by court and case type, then calculate statistics
    stats = df.groupby(['court', 'broad_case_type']).apply(
        lambda x: pd.Series({
            'total_cases': len(x),
            'resolved_within_timeline': sum(x.apply(is_resolved_within_timeline, axis=1)),
            'proportion_resolved': sum(x.apply(is_resolved_within_timeline, axis=1)) / len(x) if len(x) > 0 else 0
        })
    ).reset_index()
    
    return stats

In [78]:
def prepare_case_resolution_data(df, case_type_timelines):
    # Convert date columns to datetime
    date_columns = ['filed_date', 'activity_date']
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
    
    # Function to check if a case is resolved within timeline
    def is_resolved_within_timeline(row, case_type, timeline):
        return 1 if row['concluded'] == 1 and row['age'] <= timeline and row['broad_case_type'] == case_type else 0

    # Add a column for each case type indicating if it's resolved within timeline
    for case_type, timeline in case_type_timelines.items():
        column_name = f"{case_type}_{timeline}_days"
        df[column_name] = df.apply(
            lambda row: is_resolved_within_timeline(row, case_type, timeline), 
            axis=1
        )

    return df

In [79]:
def create_case_resolution_pivot(df, case_type_timelines):
    # Prepare the aggfunc dictionary
    aggfunc = {}
    for case_type, timeline in case_type_timelines.items():
        column = f"{case_type}_{timeline}_days"
        aggfunc[f"{column}_total"] = 'count'
        aggfunc[f"{column}_resolved"] = 'sum'
        aggfunc[f"{column}_proportion"] = lambda x: x.sum() / x.count() if x.count() > 0 else 0

    # Create pivot table
    pivot = pd.pivot_table(
        df,
        values=[f"{case_type}_{timeline}_days" for case_type, timeline in case_type_timelines.items()],
        index=['court'],
        aggfunc=aggfunc
    )

    # Rename columns to match the desired format
    new_columns = []
    for case_type, timeline in case_type_timelines.items():
        for stat in ['total', 'resolved', 'proportion']:
            new_columns.append(f"{case_type}_{timeline}_days_{stat}")
    
    pivot.columns = new_columns

    return pivot

In [ ]:

# Assuming df is your original dataframe
prepared_df = prepare_case_resolution_data(df, case_type_timelines)


In [70]:
# Example usage:
case_type_timelines = {
    'Murder': 360,
    'Criminal Revision': 90,
    'Miscellaneous Application': 90,
    'Civil Suit': 360,
    'Judicial Review': 180,
    'Constitution Petition': 180,
}

In [83]:
def create_pivot_table(df, case_type_timelines):
  # Calculate statistics using your existing function
  stats = get_case_resolution_proportions(df, case_type_timelines)

  # Pivot the data
  pivot_table = stats.pivot_table(
      index='court',
      columns='broad_case_type',
      values=['total_cases', 'resolved_within_timeline', 'proportion_resolved']
  )

  # Flatten the column hierarchy
  pivot_table.columns = [f"{col[1]}_{col[0]}" for col in pivot_table.columns]

  return pivot_table


In [84]:
# Assuming you have df and case_type_timelines
pivot_table = create_pivot_table(df, case_type_timelines)


/tmp/ipykernel_3738/877221319.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats = df.groupby(['court', 'broad_case_type']).apply(


In [ ]:
pivot_table

In [85]:
def get_case_resolution_proportions(df, case_type_timelines):
  # Convert date columns to datetime
  date_columns = ['filed_date', 'activity_date']
  for col in date_columns:
    df[col] = pd.to_datetime(df[col])

  # Filter cases based on case_type_timelines
  df = df[df['broad_case_type'].isin(case_type_timelines.keys())]

  # Function to check if a case is resolved within timeline
  def is_resolved_within_timeline(row):
    timeline = case_type_timelines.get(row['broad_case_type'], float('inf'))
    return row['concluded'] == 1 and row['age'] <= timeline

  # Group by court and case type, then calculate statistics
  stats = df.groupby(['court', 'broad_case_type']).apply(
    lambda x: pd.Series({
      'total_cases': len(x),
      'resolved_within_timeline': sum(x.apply(is_resolved_within_timeline, axis=1)),
      'proportion_resolved': sum(x.apply(is_resolved_within_timeline, axis=1)) / len(x) if len(x) > 0 else 0
    })
  ).reset_index()

  return stats


In [86]:
def create_pivot_table(df, case_type_timelines):
  # Calculate statistics using the modified function
  stats = get_case_resolution_proportions(df, case_type_timelines)

  # Pivot the data
  pivot_table = stats.pivot_table(
      index='court',
      columns='broad_case_type',
      values=['total_cases', 'resolved_within_timeline', 'proportion_resolved']
  )

  # Flatten the column hierarchy
  pivot_table.columns = [f"{col[1]}_{col[0]}" for col in pivot_table.columns]

  return pivot_table


In [87]:
# Assuming you have df and case_type_timelines
pivot_table = create_pivot_table(df, case_type_timelines)


/tmp/ipykernel_3738/1850208883.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats = df.groupby(['court', 'broad_case_type']).apply(


In [88]:
pivot_table.to_csv(f'{file_path}/reports/pmmu_resolution.csv', index=False)

In [89]:
def check_time_limit(age, case_category, concluded, time_lines):
    """
    Check if a case falls within the specified time limit for its category and is concluded.
    
    Parameters:
        age (int): The age of the case in days.
        case_category (str): The category of the case.
        concluded (int): The status of the case conclusion (1 for concluded, 0 otherwise).
        time_lines (dict): A dictionary with case categories as keys and time limits as values.
        
    Returns:
        int: 1 if the case is within the time limit and concluded, otherwise 0.
    """
    time_limit = time_lines.get(case_category, 0)
    return 1 if age <= time_limit and concluded == 1 else 0

In [90]:
# Add time lines
df['time_lines'] = df.apply(lambda row: check_time_limit(row['age'], row['broad_case_type'], row['concluded'], case_type_timelines), axis=1)

In [ ]:
df

In [91]:
pmmu_timelines = df[df['time_lines'] == 1].pivot_table(index='court', columns='broad_case_type', values='time_lines', aggfunc='count', fill_value=0)

In [92]:
total_concluded_per_court = df[df['concluded'] == 1].pivot_table(index='court', columns='broad_case_type', values='time_lines', aggfunc='count', fill_value=0)

In [93]:
resolved_within_pmmu_timeline = pmmu_timelines / total_concluded_per_court

In [96]:
resolved_within_pmmu_timeline.to_csv(f'{file_path}/reports/pmmu_timeline.csv', index=True)

In [ ]:
resolved_within_pmmu_timeline